In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import torch
from datasets import Dataset
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments, DistilBertConfig

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

/kaggle/input/sentimentalaysis/final_data.csv


In [2]:
data = pd.read_csv("/kaggle/input/sentimentalaysis/final_data.csv")

In [3]:
data = data

In [4]:
# !pip install transformers

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(data["text"], data["labels"], test_size=0.1, random_state=42)

In [5]:

from transformers import AutoTokenizer, DistilBertForSequenceClassification

In [6]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [7]:
X_train = [str(i) for i in X_train]
X_test = [str(i) for i in X_test]

In [8]:
max_len = 128

In [9]:
train_encodings = tokenizer(X_train, truncation=True , padding='max_length', max_length=max_len)
test_encodings = tokenizer(X_test,truncation=True, padding='max_length', max_length=max_len)

In [12]:
Y_train.shape

(187032,)

In [13]:
len(test_encodings["input_ids"][0])

128

In [14]:
train_encodings[1]

Encoding(num_tokens=55, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    Y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    Y_test
))

In [17]:
train_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(55,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(55,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [96]:
test_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(43,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(43,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [11]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [12]:
training_args = TFTrainingArguments(   
    output_dir="kaggle/working/weights",
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    eval_steps=20000,
    report_to=None,
    save_total_limit=2,
    logging_dir='./logs', 
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [13]:
config = DistilBertConfig(
    max_position_embeddings = 128,
    n_heads=8,
    n_layers=4,
)

In [14]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", config, num_labels = 28)

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

In [17]:
trainer.evaluate(test_dataset)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


{'eval_loss': 1.8899130641358737}

In [18]:
save_directory = "/saved_models" 

model.save_pretrained(save_directory)

In [20]:
tokenizer.save_pretrained(save_directory)

('/saved_models/tokenizer_config.json',
 '/saved_models/special_tokens_map.json',
 '/saved_models/vocab.txt',
 '/saved_models/added_tokens.json',
 '/saved_models/tokenizer.json')

In [21]:
model.save_pretrained(save_directory)

In [23]:
tokenizer_fine_tuned = AutoTokenizer.from_pretrained(save_directory)

model_fine_tuned = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

Some layers from the model checkpoint at /saved_models were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /saved_models and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [83]:
test_text = X_test[11100]

test_text

'I mean it’s a default subreddit soooo... not really saying much lol'

In [73]:
test_text = "is suprised"

In [78]:
test_text

'is suprised'

In [85]:
Y_test.iloc[11100]

4

In [86]:
predict_input = tokenizer_fine_tuned.encode(
    test_text,
    truncation = True,
    padding = "max_length",
    max_length = 128,
    return_tensors = 'tf'    
)

In [87]:
predict_input

<tf.Tensor: shape=(1, 128), dtype=int32, numpy=
array([[  101,  1045,  2812,  2009,  1521,  1055,  1037, 12398,  4942,
         5596, 23194, 17111,  9541,  1012,  1012,  1012,  2025,  2428,
         3038,  2172,  8840,  2140,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [88]:
output = model_fine_tuned(predict_input)[0]

In [89]:
output

<tf.Tensor: shape=(1, 28), dtype=float32, numpy=
array([[-1.0996838 ,  1.9489961 , -2.1040127 , -0.24600585,  1.4081815 ,
        -2.9567783 , -0.16169839, -2.3757887 , -2.5556593 , -0.9335374 ,
         1.3470159 , -2.1806698 , -2.6056263 , -3.1674876 , -2.613934  ,
        -3.831134  , -5.600054  , -2.3141367 , -2.8637645 , -3.7837334 ,
         3.564959  , -1.4236544 , -3.6499984 , -0.10755472, -4.205346  ,
        -3.5211074 , -1.8668207 , -3.270887  ]], dtype=float32)>

In [90]:
prediction_value = tf.argmax(output, axis = 1).numpy()[0]

prediction_value

20